In [50]:
# Load environment variables
from dotenv import load_dotenv

load_dotenv()

True

In [51]:
# Setup a document loader
from langchain.document_loaders import TextLoader

loader = TextLoader("story_1.txt")
docs = loader.load()
print(docs[0].page_content)

In the realm of Eldoria, a land of sprawling kingdoms and intricate alliances, a cold wind whispered secrets of impending turmoil. Houses of noble lineage vied for power, their ambitions woven into a tapestry of politics, betrayal, and epic battles.

At the heart of it all was House Arion, ruling from their majestic fortress atop the Frostpeak Mountains. Led by Lord Cedric Arion, a cunning and shrewd strategist, the House had managed to maintain their grip on the realm despite growing tensions. Yet, beneath the surface, rivalries brewed as they always did.

House Valerian, known for their mastery of arcane arts, coveted the coveted Dragonheart Relic, a gem rumored to grant immense power to its possessor. The Valerians, led by Lady Elara Valerian, plotted to steal the relic from the Arions, plunging Eldoria into chaos.

Amidst the turmoil, a bastard son of House Arion named Eamon Snow, skilled with a blade and burdened by his illegitimate birth, found himself torn between loyalty to his

In [52]:
# Split the text
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=100,
    chunk_overlap=20,
    length_function=len,
    is_separator_regex=False
)

texts = text_splitter.split_documents(docs)
texts[0].page_content

'In the realm of Eldoria, a land of sprawling kingdoms and intricate alliances, a cold wind whispered'

In [53]:
# Create an embedding
from langchain.embeddings import OpenAIEmbeddings

embeddings = OpenAIEmbeddings()

In [54]:
# Create a vector store using meilisearch
from langchain.vectorstores import Meilisearch

# persist_directory = 'db'
# vectordb = Chroma.from_documents(documents=texts, embedding=embeddings, persist_directory=persist_directory)
vectordb = Meilisearch.from_documents(documents=texts, embedding=embeddings, index_name='quipbot-test')

In [ ]:
# Search in our vector store
query = "Who was the lord of House Arion?"
docs = vectordb.similarity_search(query)
print(docs[0].page_content)

In [ ]:
# Create a retriever from the vector db
retriever = vectordb.as_retriever()
retriever.get_relevant_documents(query=query)

In [ ]:
# Create a conversation agent
from langchain.chains.conversation.memory import ConversationBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA

llm = ChatOpenAI(model='gpt-3.5-turbo', temperature=0, max_tokens=500)
memory = ConversationBufferMemory()
# conversation = ConversationalRetrievalChain.from_llm(llm=llm, retriever=retriever, chain_type='stuff')
qa_chain = RetrievalQA.from_chain_type(llm=llm, retriever=retriever)

In [ ]:
# Test the retriever
response = qa_chain(query)
# response = conversation.predict(input="What is my name?")
response['result']